# Tool Use and Function Calling

## Empowering Agents with Tools

In this notebook, we'll master:

1. **Creating Tools**: Building custom tools for agents
2. **Async Tools**
3. **MCP Tools**
4. **Structured Outputs**

Tools transform agents from conversational entities into action-oriented problem solvers!

## Setup

In [2]:
# Import required libraries
import os
import json
import asyncio
import httpx
from typing import Dict, List, Any, Optional
from datetime import datetime, timedelta
import random

# AutoGen imports
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
import tempfile

In [ ]:
# Set up model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    # api_key=os.environ.get("OPENAI_API_KEY"),
)

## 1. Basic Tool Creation

Let's start with simple tools and understand how they work. 

[The AssistantAgent automatically converts Python functions into FunctionTool objects, which can be used as a tool by the agent.](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/agents.html#using-tools-and-workbench:~:text=Function%20Tool%23,is%20automatically%20generated.) 

FunctionTool automatically generates the tool schema from the function signature and docstring.

below is a simple example where the schema is automatically generated for the function:

In [4]:
# source: https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/agents.html#using-tools-and-workbench
from autogen_core.tools import FunctionTool


# Define a tool using a Python function.
async def web_search_func(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


# This step is automatically performed inside the AssistantAgent if the tool is a Python function.
web_search_function_tool = FunctionTool(web_search_func, description="Find information on the web")
# The schema is provided to the model during AssistantAgent's on_messages call.
web_search_function_tool.schema


{'name': 'web_search_func',
 'description': 'Find information on the web',
 'parameters': {'type': 'object',
  'properties': {'query': {'description': 'query',
    'title': 'Query',
    'type': 'string'}},
  'required': ['query'],
  'additionalProperties': False},
 'strict': False}

In [5]:
# Simple calculation tools
def calculate_compound_interest(
    principal: float,
    rate: float,
    time: int,
    compounds_per_year: int = 12
) -> Dict[str, float]:
    """Calculate compound interest.
    
    Args:
        principal: Initial amount
        rate: Annual interest rate (as percentage, e.g., 5 for 5%)
        time: Time period in years
        compounds_per_year: Number of times interest compounds per year
    
    Returns:
        Dictionary with final amount and interest earned
    """
    rate_decimal = rate / 100
    amount = principal * (1 + rate_decimal/compounds_per_year) ** (compounds_per_year * time)
    interest = amount - principal
    
    return {
        "principal": principal,
        "rate": rate,
        "time": time,
        "final_amount": round(amount, 2),
        "interest_earned": round(interest, 2)
    }

def convert_currency(
    amount: float,
    from_currency: str,
    to_currency: str
) -> Dict[str, Any]:
    """Convert currency from one type to another.
    
    Args:
        amount: Amount to convert
        from_currency: Source currency code (e.g., 'USD')
        to_currency: Target currency code (e.g., 'EUR')
    
    Returns:
        Dictionary with conversion details
    """
    # Mock exchange rates for demonstration
    rates = {
        "USD": 1.0,
        "EUR": 0.92,
        "GBP": 0.79,
        "JPY": 149.50,
        "CAD": 1.36,
    }
    
    if from_currency not in rates or to_currency not in rates:
        return {"error": "Currency not supported"}
    
    # Convert to USD first, then to target currency
    usd_amount = amount / rates[from_currency]
    converted = usd_amount * rates[to_currency]
    
    return {
        "original_amount": amount,
        "from_currency": from_currency,
        "to_currency": to_currency,
        "converted_amount": round(converted, 2),
        "exchange_rate": round(rates[to_currency] / rates[from_currency], 4)
    }

# Create an agent with these tools
financial_agent = AssistantAgent(
    name="financial_advisor",
    model_client=model_client,
    tools=[calculate_compound_interest, convert_currency],
    system_message="""You are a financial advisor. Use the available tools to help with financial calculations.
    Always explain your calculations clearly.""",
)

# Test the agent with tools
result = await financial_agent.run(
    task="I have $10,000. How much will it grow to in 5 years at 7% annual interest? Also, convert the final amount to EUR."
)

print("Financial Agent Response:")
for msg in result.messages:
    if msg.source == "financial_advisor" and msg.content:
        print(msg.content)

Financial Agent Response:
[FunctionCall(id='call_NfAqQLlyC3h8voDhXmLZDlWf', arguments='{"principal":10000,"rate":7,"time":5}', name='calculate_compound_interest')]
[FunctionExecutionResult(content="{'principal': 10000.0, 'rate': 7.0, 'time': 5, 'final_amount': 14176.25, 'interest_earned': 4176.25}", name='calculate_compound_interest', call_id='call_NfAqQLlyC3h8voDhXmLZDlWf', is_error=False)]
{'principal': 10000.0, 'rate': 7.0, 'time': 5, 'final_amount': 14176.25, 'interest_earned': 4176.25}


## 2. Async Tools

Many real-world tools need to be asynchronous (API calls, database queries, etc.):

In [6]:
# Async tool examples
async def fetch_stock_price(symbol: str) -> Dict[str, Any]:
    """Fetch current stock price (simulated).
    
    Args:
        symbol: Stock symbol (e.g., 'AAPL', 'GOOGL')
    
    Returns:
        Dictionary with stock information
    """
    # Simulate API delay
    await asyncio.sleep(0.5)
    
    # Mock stock data
    stocks = {
        "AAPL": {"price": 178.25, "change": 2.15, "change_percent": 1.22},
        "GOOGL": {"price": 139.80, "change": -0.95, "change_percent": -0.67},
        "MSFT": {"price": 378.50, "change": 3.20, "change_percent": 0.85},
        "AMZN": {"price": 145.60, "change": 1.80, "change_percent": 1.25},
    }
    
    if symbol.upper() not in stocks:
        return {"error": f"Stock symbol {symbol} not found"}
    
    data = stocks[symbol.upper()]
    return {
        "symbol": symbol.upper(),
        "price": data["price"],
        "change": data["change"],
        "change_percent": data["change_percent"],
        "timestamp": datetime.now().isoformat()
    }

async def analyze_portfolio(
    holdings: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """Analyze a stock portfolio.
    
    Args:
        holdings: List of holdings, each with 'symbol' and 'shares'
    
    Returns:
        Portfolio analysis
    """
    total_value = 0
    portfolio_details = []
    
    for holding in holdings:
        stock_data = await fetch_stock_price(holding["symbol"])
        if "error" not in stock_data:
            value = stock_data["price"] * holding["shares"]
            total_value += value
            portfolio_details.append({
                "symbol": holding["symbol"],
                "shares": holding["shares"],
                "current_price": stock_data["price"],
                "value": round(value, 2),
                "daily_change": round(stock_data["change"] * holding["shares"], 2)
            })
    
    return {
        "total_value": round(total_value, 2),
        "holdings": portfolio_details,
        "analysis_time": datetime.now().isoformat()
    }

# Create a stock market agent
market_agent = AssistantAgent(
    name="market_analyst",
    model_client=model_client,
    tools=[fetch_stock_price, analyze_portfolio],
    system_message="""You are a stock market analyst. Use the tools to provide market insights.
    Format responses clearly with relevant data.""",
)

# Test async tools
result = await market_agent.run(
    task="""Check the current prices of AAPL and MSFT. 
    Then analyze a portfolio with 100 shares of AAPL and 50 shares of MSFT."""
)

print("Market Analysis:")
for msg in result.messages:
    if msg.source == "market_analyst" and msg.content:
        print(msg.content[:1000])  # Truncate for readability

Market Analysis:
[FunctionCall(id='call_sqZkCx4oImicT0P0dEMVk5HR', arguments='{"symbol": "AAPL"}', name='fetch_stock_price'), FunctionCall(id='call_tZpnYHve601mSUL2SNTOUdDZ', arguments='{"symbol": "MSFT"}', name='fetch_stock_price'), FunctionCall(id='call_GXySIGvO7T7gjl2BLsu5WIQ6', arguments='{}', name='analyze_portfolio')]
[FunctionExecutionResult(content="{'symbol': 'AAPL', 'price': 178.25, 'change': 2.15, 'change_percent': 1.22, 'timestamp': '2025-08-20T16:05:44.883814'}", name='fetch_stock_price', call_id='call_sqZkCx4oImicT0P0dEMVk5HR', is_error=False), FunctionExecutionResult(content="{'symbol': 'MSFT', 'price': 378.5, 'change': 3.2, 'change_percent': 0.85, 'timestamp': '2025-08-20T16:05:44.883934'}", name='fetch_stock_price', call_id='call_tZpnYHve601mSUL2SNTOUdDZ', is_error=False), FunctionExecutionResult(content='1 validation error for analyze_portfolioargs\nholdings\n  Field required [type=missing, input_value={}, input_type=dict]\n    For further information visit https://er

## 3. Model Context Protocol (MCP) Workbench

The AssistantAgent is compatible with mcp (model context protocol) tools.

In [7]:
# source: https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/agents.html#using-tools-and-workbench
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.mcp import McpWorkbench, StdioServerParams

# Get the fetch tool from mcp-server-fetch.
fetch_mcp_server = StdioServerParams(command="uvx", args=["mcp-server-fetch"])

# Create an MCP workbench which provides a session to the mcp server.
async with McpWorkbench(fetch_mcp_server) as workbench:  # type: ignore
    # Create an agent that can use the fetch tool.
    model_client = OpenAIChatCompletionClient(model="gpt-4.1-nano")
    fetch_agent = AssistantAgent(
        name="fetcher", model_client=model_client, workbench=workbench, reflect_on_tool_use=True
    )

    # Let the agent fetch the content of a URL and summarize it.
    result = await fetch_agent.run(task="Summarize the content of https://en.wikipedia.org/wiki/Seattle")
    assert isinstance(result.messages[-1], TextMessage)
    print(result.messages[-1].content)

    # Close the connection to the model client.
    await model_client.close()

Seattle is a major city located in the state of Washington, United States. It was founded on November 13, 1851, and officially incorporated as a town on January 14, 1865, and as a city on December 2, 1869. The city is named after Chief Seattle.

Geographically, Seattle covers an area of approximately 142 square miles, with about 84 square miles of land and 58 square miles of water. It sits at an elevation of 148 feet. The city is situated in King County and has a population of around 737,000 as of the 2020 census, with an estimated increase to over 780,000 in 2024. It ranks as the 54th most populous city in North America and 18th in the United States.

Seattle is known for landmarks such as the Space Needle, Pike Place Market, and the Amazon Spheres. It is often called "The Emerald City," "Jet City," and "Rain City." The city's government operates under a mayor-council system, with Bruce Harrell serving as mayor. Seattle has a vibrant urban and metropolitan area, with a population of o

## 4. Structured Outputs

In [8]:
from autogen_agentchat.messages import StructuredMessage
from typing import Literal
from pydantic import BaseModel
from autogen_agentchat.ui import Console


# The response format for the agent as a Pydantic base model.
class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["math", "coding", "other"]


# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(model="gpt-4o")
agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    system_message="Categorize the problems given by the user as: math, coding, other.",
    # Define the output content type of the agent.
    output_content_type=AgentResponse,
)

result = await Console(agent.run_stream(task="What is the sum of 1 and 2?"))

# Check the last message in the result, validate its type, and print the thoughts and response.
assert isinstance(result.messages[-1], StructuredMessage)
assert isinstance(result.messages[-1].content, AgentResponse)
print("Thought: ", result.messages[-1].content.thoughts)
print("Response: ", result.messages[-1].content.response)
await model_client.close()

---------- TextMessage (user) ----------
What is the sum of 1 and 2?
---------- StructuredMessage[AgentResponse] (assistant) ----------
{"thoughts":"This problem is asking for a numerical calculation. Specifically, it's about adding two numbers together. Therefore, this falls under elementary arithmetic, which is a category of mathematics.","response":"math"}
Thought:  This problem is asking for a numerical calculation. Specifically, it's about adding two numbers together. Therefore, this falls under elementary arithmetic, which is a category of mathematics.
Response:  math


In [11]:
from autogen_agentchat.messages import StructuredMessage
from typing import Literal
from pydantic import BaseModel
from autogen_agentchat.ui import Console


# The response format for the agent as a Pydantic base model.
class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["negative", "positive", "neutral"]


# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(model="gpt-4o")
agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    system_message="Categorize the sentences as: negative, positive, neutral.",
    # Define the output content type of the agent.
    output_content_type=AgentResponse,
)

result = await Console(agent.run_stream(task="I love teaching people about agents!"))

# Check the last message in the result, validate its type, and print the thoughts and response.
assert isinstance(result.messages[-1], StructuredMessage)
assert isinstance(result.messages[-1].content, AgentResponse)
print("Thought: ", result.messages[-1].content.thoughts)
print("Response: ", result.messages[-1].content.response)
await model_client.close()

---------- TextMessage (user) ----------
I love teaching people about agents!
---------- StructuredMessage[AgentResponse] (assistant) ----------
{"thoughts":"The sentence expresses a positive emotion towards teaching people about agents.","response":"positive"}
Thought:  The sentence expresses a positive emotion towards teaching people about agents.
Response:  positive


## 5. Agents as Tools

You can also use agents themselves as tools.

In [9]:
content_to_summarize = """
'Modern LLM agents are goal-directed systems built around large language models that can plan and act, 
not just chat. They interact with external tools, services, and environments to achieve outcomes, 
iterating through a loop of observing context, planning next steps, taking actions, and using the 
results to refine their approach. Architecturally, they pair an LLM with a function-calling layer for 
tools and APIs, a memory or state store, and policies that constrain behavior. This enables multi-step 
workflows such as analyzing datasets, booking travel, triaging support tickets, or orchestrating business 
processes end to end.\n\nTo stay grounded, agents often augment the model with retrieval so it can consult 
documents, knowledge bases, or the web before answering. Tool use spans calling APIs and databases, browsing, 
and writing or executing code in a sandbox to transform data or validate results. Many implementations 
decompose tasks into subgoals, track intermediate state, and emit structured outputs for downstream systems. 
Orchestrators can coordinate multiple specialized agents (for example, researcher, coder, tester) and hand 
off work, while guardrails enforce schemas, permissions, and organizational policies. Integration is 
typically done through structured function calls, JSON schemas, and event-driven workflows.\n\nDespite 
their versatility, LLM agents have limits: they can hallucinate, mishandle edge cases, and struggle with 
very long-horizon plans or ambiguous goals. Practical deployments rely on safeguards such as permissioning, 
human-in-the-loop review, sandboxed tool execution, rate limits, and continuous evaluation of task success 
and safety. Cost and latency are important, so designers cache results, reuse context, and adapt model size 
to task complexity. Good applications align with agent strengths—information synthesis, routine workflow 
automation, data wrangling, and software assistance—while avoiding high-stakes decisions without oversight. 
Looking ahead, better memory, more reliable planning, and tighter integration with enterprise systems and 
hardware will make agents more capable and trustworthy.'
"""

In [10]:
# source: https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.tools.html#autogen_agentchat.tools.AgentTool
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.tools import AgentTool
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(model="gpt-4.1")
writer = AssistantAgent(
    name="summarizer",
    description="You take in text and you summarize.",
    model_client=model_client,
    system_message="Write well.",
)
summarizer_tool = AgentTool(agent=writer)

# Create model client with parallel tool calls disabled for the main agent
main_model_client = OpenAIChatCompletionClient(model="gpt-4.1", parallel_tool_calls=False)
assistant = AssistantAgent(
    name="assistant",
    model_client=main_model_client,
    tools=[summarizer_tool],
    system_message="You are a helpful assistant. When requested to summarize information, use the summarizer tool.",
)
await Console(assistant.run_stream(task=f"""
                                   Summarize this: 
                                   {content_to_summarize}
                                   """))

---------- TextMessage (user) ----------

                                   Summarize this: 
                                   
'Modern LLM agents are goal-directed systems built around large language models that can plan and act, 
not just chat. They interact with external tools, services, and environments to achieve outcomes, 
iterating through a loop of observing context, planning next steps, taking actions, and using the 
results to refine their approach. Architecturally, they pair an LLM with a function-calling layer for 
tools and APIs, a memory or state store, and policies that constrain behavior. This enables multi-step 
workflows such as analyzing datasets, booking travel, triaging support tickets, or orchestrating business 
processes end to end.

To stay grounded, agents often augment the model with retrieval so it can consult 
documents, knowledge bases, or the web before answering. Tool use spans calling APIs and databases, browsing, 
and writing or executing code in a sa

TaskResult(messages=[TextMessage(id='80a3a530-3c31-43ec-abc4-974a843bf726', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 20, 15, 19, 38, 807484, tzinfo=datetime.timezone.utc), content="\n                                   Summarize this: \n                                   \n'Modern LLM agents are goal-directed systems built around large language models that can plan and act, \nnot just chat. They interact with external tools, services, and environments to achieve outcomes, \niterating through a loop of observing context, planning next steps, taking actions, and using the \nresults to refine their approach. Architecturally, they pair an LLM with a function-calling layer for \ntools and APIs, a memory or state store, and policies that constrain behavior. This enables multi-step \nworkflows such as analyzing datasets, booking travel, triaging support tickets, or orchestrating business \nprocesses end to end.\n\nTo stay grounded, agents often augme

In [17]:
# Complex tool combining multiple operations
async def analyze_webpage(
    url: str,
    extract_type: str = "summary"
) -> Dict[str, Any]:
    """Analyze a webpage and extract information.
    
    Args:
        url: URL to analyze
        extract_type: Type of extraction ('summary', 'links', 'metadata')
    
    Returns:
        Extracted information
    """
    # Simulate web scraping
    await asyncio.sleep(0.5)
    
    # Mock data based on URL
    if "github.com" in url:
        if extract_type == "summary":
            return {
                "url": url,
                "title": "AutoGen - Multi-agent Framework",
                "summary": "A framework for building multi-agent AI applications",
                "main_topics": ["AI agents", "LLM integration", "Tool use"],
                "word_count": 1250
            }
        elif extract_type == "links":
            return {
                "url": url,
                "total_links": 42,
                "internal_links": 28,
                "external_links": 14,
                "top_domains": ["github.com", "microsoft.com", "openai.com"]
            }
        elif extract_type == "metadata":
            return {
                "url": url,
                "last_updated": "2024-01-15",
                "language": "en",
                "technologies": ["Python", "TypeScript"],
                "license": "MIT"
            }
    
    return {
        "url": url,
        "error": "Could not analyze webpage"
    }

def compare_webpages(
    page1_data: Dict[str, Any],
    page2_data: Dict[str, Any]
) -> Dict[str, Any]:
    """Compare data from two webpages.
    
    Args:
        page1_data: Data from first webpage
        page2_data: Data from second webpage
    
    Returns:
        Comparison results
    """
    comparison = {
        "pages_compared": [page1_data.get("url"), page2_data.get("url")],
        "similarities": [],
        "differences": []
    }
    
    # Find common keys
    common_keys = set(page1_data.keys()) & set(page2_data.keys())
    
    for key in common_keys:
        if page1_data[key] == page2_data[key]:
            comparison["similarities"].append({
                "field": key,
                "value": page1_data[key]
            })
        else:
            comparison["differences"].append({
                "field": key,
                "page1": page1_data[key],
                "page2": page2_data[key]
            })
    
    return comparison

# Create a web analyst agent
web_agent = AssistantAgent(
    name="web_analyst",
    model_client=model_client,
    tools=[analyze_webpage, compare_webpages],
    system_message="""You are a web analyst. Use the tools to analyze and compare webpages.
    Provide insights based on the extracted data.""",
)

# Test complex tool usage
result = await web_agent.run(
    task="""Analyze https://github.com/microsoft/autogen - extract both summary and metadata.
    Then analyze https://github.com/langchain-ai/langchain and compare the two pages."""
)

print("Web Analysis:")
for msg in result.messages:
    if msg.source == "web_analyst" and msg.content:
        print(msg.content[:1000])

Web Analysis:
[FunctionCall(id='call_JCt3TSziUpciiUpz27gqaMqY', arguments='{"url": "https://github.com/microsoft/autogen", "extract_type": "summary"}', name='analyze_webpage'), FunctionCall(id='call_TDuYfp6hkqwtFumQsqxyia2O', arguments='{"url": "https://github.com/microsoft/autogen", "extract_type": "metadata"}', name='analyze_webpage'), FunctionCall(id='call_7Ta9HoDZGwel919djanhfO85', arguments='{"url": "https://github.com/langchain-ai/langchain", "extract_type": "summary"}', name='analyze_webpage'), FunctionCall(id='call_qTzFzbY2WCXfYN2qefEemI7z', arguments='{"url": "https://github.com/langchain-ai/langchain", "extract_type": "metadata"}', name='analyze_webpage')]
[FunctionExecutionResult(content="{'url': 'https://github.com/microsoft/autogen', 'title': 'AutoGen - Multi-agent Framework', 'summary': 'A framework for building multi-agent AI applications', 'main_topics': ['AI agents', 'LLM integration', 'Tool use'], 'word_count': 1250}", name='analyze_webpage', call_id='call_JCt3TSziUpc

## Clean Up

In [18]:
# Close the model client
await model_client.close()
print("Model client closed successfully.")

Model client closed successfully.


# Summary
In this notebook, you learned how to empower agents with tools using the AutoGen framework:


1. ✅ **Creating Tools**: How to define both synchronous and asynchronous Python functions as tools for agents.
2. ✅ **Async Tools**: How to create async tools
3. ✅ **MCP Tools**: How to create mcp compatible tools
4. ✅ **Structured Outputs**: How to get structured outputs from agents
5. ✅ **Agents as Tools**: How to convert agents into tools for other agents


## Key Takeaways

- **Tools extend agent capabilities**: By registering Python functions or even other agents as tools, you enable agents to take real actions and solve practical problems.
- **Type hints and docstrings matter**: They drive schema generation and improve tool usability for LLMs.
- **Async support is essential**: Many real-world tools (APIs, I/O) are asynchronous—AutoGen supports both sync and async tools.
- **Leverage MCP for a more robust system**
- **Leverage agents as tools and structured outputs to create controllable agentic workflows**

# Exercises

1. **Exercise 1**: Create a tool that calls a real external API (e.g., weather, news, or finance).
2. **Exercise 2**: Implement an agent as a tool for a financial data analysis scenario.